In [14]:
import yfinance as yf
from datetime import date,timedelta
from pycaret.time_series import *
import plotly.express as px
import warnings
import pandas as pd

In [6]:
stock_name = ["AKBNK.IS"]
yday = date.today() - timedelta(days=1)
startday = yday - timedelta(days=729)
stock_data = yf.download(tickers = stock_name[0], start = startday, end = yday)
df = stock_data.copy()

[*********************100%***********************]  1 of 1 completed


In [7]:
start_date = df.index.min()
end_date = df.index.max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
complete_df = pd.DataFrame(index=date_range)
#Create missing days (holiday, weekends etc.) and linear fill
merged_df = complete_df.merge(df, left_index=True, right_index=True, how='left')
# Maybe time, quadratic or from_derivatives
filled_df = merged_df.interpolate(method='linear')

In [8]:
fnl_data = filled_df["Close"]

In [9]:
exp_name = setup(data = fnl_data,  fh = 1)

,Description,Value
0,session_id,5750
1,Target,Close
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(723, 1)"
5,Transformed data shape,"(723, 1)"
6,Transformed train set shape,"(722, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [10]:
arima = create_model('arima')

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2023-06-23,1.6235,0.9840,0.5008,0.5008,0.0250,0.0254
1,2023-06-24,0.3001,0.1821,0.0928,0.0928,0.0046,0.0046
2,2023-06-25,0.4740,0.2878,0.1466,0.1466,0.0072,0.0072
Mean,NaT,0.7992,0.4846,0.2467,0.2467,0.0123,0.0124
SD,NaT,0.5872,0.3557,0.1810,0.1810,0.0091,0.0092


In [11]:
_ = predict_model(arima, verbose=False)
z = pull()

In [12]:
tuned_arima = tune_model(arima, verbose=False)

In [13]:
_ = predict_model(tuned_arima, verbose=False)
zz = pull()

In [15]:
finalized = finalize_model(tuned_arima)

In [16]:
finalized

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('model',
                                                                ARIMA(order=(1,
                                                                             1,
                                                                             1),
                                                                      seasonal_order=(1,
                                                                                      0,
                                                                                      0,
                                                                                      2)))]))])

In [1]:
from yahooquery import Ticker

In [32]:
stocks = ["TRY=X","XU100.IS","AKBNK.IS","ALBRK.IS","GARAN.IS","ICBCT.IS","SKBNK.IS","TSKB.IS","HALKB.IS","ISCTR.IS","VAKBN.IS","YKBNK.IS"]
yday = date.today() - timedelta(days=1)
startday = yday - timedelta(days=729)
tickers = Ticker(" ".join(stocks), asynchronous=True)

In [33]:
df = tickers.history(start=startday, end=yday)
df_close = df.reset_index().pivot(index='date',
                                  columns='symbol', values='close')
df_adj_close = df.reset_index().pivot(index='date',
                                  columns='symbol', values='adjclose')

In [39]:
#define dictionary
some_dict = {'TRY=X': 'USDTRY',
             'XU100.IS': 'BIST100'}
#rename columns in DataFrame using dictionary
df_close.rename(columns=some_dict, inplace=True)
df_adj_close.rename(columns=some_dict, inplace=True)
df_close.columns = [col.replace('.IS', '') for col in df_close.columns]
df_adj_close.columns = [col.replace('.IS', '') for col in df_adj_close.columns]
df_adj_close.tail(10)

,AKBNK,ALBRK,GARAN,HALKB,ICBCT,ISCTR,SKBNK,USDTRY,TSKB,VAKBN,BIST100,YKBNK
date,,,,,,,,,,,,
2023-06-16,18.889999,2.83,30.080000,12.88,7.59,13.42,2.57,23.650801,4.38,10.19,5475.500000,12.80
2023-06-19,18.510000,2.78,29.420000,12.22,7.27,12.88,2.42,23.548500,4.21,9.63,5269.299805,12.19
2023-06-20,19.040001,2.76,30.660000,12.05,7.28,13.26,2.48,23.601500,4.26,9.46,5251.600098,12.44
2023-06-21,19.309999,2.83,30.100000,11.79,7.30,13.10,2.48,23.572100,4.21,9.27,5186.799805,12.45
2023-06-22,19.190001,2.89,30.360001,12.02,7.38,13.24,2.51,23.554800,4.26,9.47,5428.299805,12.41
2023-06-23,19.790001,2.93,31.600000,12.38,7.62,13.65,2.58,24.885099,4.42,9.80,5582.799805,12.84
2023-06-26,20.420000,2.97,32.500000,12.68,7.74,14.05,2.67,25.317801,4.51,10.05,5732.899902,13.13
2023-06-27,20.200001,2.94,32.040001,12.48,7.70,14.06,2.64,25.960159,4.48,9.91,5759.100098,12.98
2023-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.989990,NaN,NaN,NaN,NaN
